##  Related articles
+ [Create your own VM on Google Cloud Platform](/vm_on_gpc.html)
+ [Copy paste over VNC](/copy_paste_over_vnc.html)
+ [Accessing jupyter running on VM](/jupyter_on_vm.html)

# Connection to the desktop of cloud VM over VNC

By default, the most convenient way of accessing your VM is through web based SSH connection provided by GCP, all you need to do is just click on **SHH** name by your instance. 
Then new window will open, and after some time you will see prompt line of your instance:
<img class="img-center" src="../../images/vnc/prompt01.PNG" alt="prompt01" style="width: 400px"/>
You can wander if your instance is not protected since you did not provide any password - actually it is secured quite well, and login was done over SSH channel utilizing embedded SSH-RSA key. 

In previous [article](/vm_on_gpc.html) I also described a way to access your VM using *putty* .

Ok, in order to have fully functional VM, which would give you a feeling of real computer (only remote) we need remote access to its GUI interface (desktop). Here I will describe how to access your VM using popular VNC client.

Unfortunately, it is not so straight forward, since there is some packages that needs to be installed, and some configurations has to be created, but hopefully using this manual it will be a bit easier.

## VNC server installation on VM

I was inspired by other [article](https://www.digitalocean.com/community/tutorials/how-to-install-and-configure-vnc-on-debian-9) describing VNC connection on Debian systems.

Here I am assuming that you have VM created as in the [article](/vm_on_gpc.html) I described before.

First, in the SSH console please input following commands, in order to install Xfce desktop environment on our VM:

```python
sudo apt update
sudo apt install xfce4 xfce4-goodies
```
During installation select English, then Enter key.

In case of problem during installation, try:
```python
ps aux | grep -i apt
sudo kill <process id> or
sudo dpkg --configure -a
```

After successful installation of Xfce desktop, it is time to install VNC server:

```python
sudo apt install vnc4server
```

After installation we need to run  *vncserver* in order to finish its settings:
```python
sudo vncserver
```
Here you will be asked to provide password for remote VNC service (through VNC viewer on your local computer). Then repeat password, and refuse creation of the password for View only 
```python
ptwnuk@myinstance03:~$ sudo vncserver
You will require a password to access your desktops.
Password:
Verify:
Would you like to enter a view-only password (y/n)? n
/usr/bin/xauth:  file /root/.Xauthority does not exist
New 'myinstance03.europe-west4-a.c.white-collector-222018.internal:1 (root)' desktop at :1 on machine myinstance03.
europe-west4-a.c.white-collector-222018.internal
Starting applications specified in /etc/X11/Xvnc-session
Log file is /root/.vnc/myinstance03.europe-west4-a.c.white-collector-222018.internal:1.log
Use xtigervncviewer -SecurityTypes VncAuth -passwd /root/.vnc/passwd :1 to connect to the VNC server.
```

Now you can check if your VNC server is already running on your VM:
You can list VNC sessions using:
```python
vncserver -list
```

Now we need to change VNC configuration file, in order to make sure, that whenever VNC is started it should execute Xfce desktop.
But before editing file, we must kill VNC process, which stared during our initialization procedure above. For that type the following:

```python
ptwnuk@myistance:~$ vncserver -kill :1
Killing Xtigervnc process ID 18595... success!
```

If there is already configuration file for VNC server then lets make a copy of it:
```python
mv ~/.vnc/xstartup ~/.vnc/xstartup.bak
nano ~/.vnc/xstartup
```
Then inside the file past:
```python
~/.vnc/xstartup
#!/bin/bash
xrdb $HOME/.Xresources
startxfce4 &
```
Then press Ctrl+X, then confirm saving file.
```python
sudo chmod +x ~/.vnc/xstartup
vncserver
```
Now your VNC server should be running! The only missing component is connection from our local VNC viewer computer to remote VNC server on VM.

For that we will use *putty* connection over SSH, with VNC ports tunneling ([here](/vm_on_gpc.html) you can read how to enable such connection).

### VNC port tunnelling

First we need to enable port used by VNC to be tunneled over SSH, which can be configured in putty. First close your previous SSH putty connection, and open putty configuration window. Then load your previously saved session, so you can edit it. Proper configuration for port tunneling is shown on image below:
<img class="img-center" src="../../images/vnc/putty07.PNG" style="width: 400px"/>
then click add button, and one more time save updated session.
Now you can open connection one more time to your VM using putty.
The last step we need to make is to run vncserver on VM, so in command line just type:
```python 
vncserver```

### Connection with VNC Viewer to VM
First you need to install VNC viewer, which you can find [here](https://www.realvnc.com/en/connect/download/viewer/).

In the VNC Viewer click File/New Connection - where you can specify new connection. In the field VNC server provide: loacalhost:5901 - this address over SSH tunneling should finally reach our remote VNC server. Then add any name for that newly created connection and then click Ok button. 
Now just double click on new connection - to open VNC connection to your remote computer. Authentication window will appear, where you need to provide a password which you choose during configuration of vncserver on your remote computer. You can also select "Remember password" for future connections.

If everything was configured properly, you should already see desktop of your remote virtual machine - congratulations!

In some cases, colors could look unnatural, this is due to connection speed optimization. If your connection is of high quality and your both machines are fast enough, you can improve your color rendering by modifying connection parameters in the VNC Viewer. Just click with right button on the title top bar of your VNC connection window, and then select Properties. Go to Expert settings, and modify ColorLevel property with the following value "pal8" or "full" depending on your connection, and then click Ok.
<img class="img-center" src="../../images/vnc/putty08.PNG" style="width: 400px"/>

After correction of color schema, your desktop should look similar to this:
<img class="img-center" src="../../images/vnc/putty09.PNG" style="width: 400px"/>
Now you can fully enjoy a reliable connection to your virtual machine desktop :)

## Some customization
To fully enjoy your remote desktop, you should install some web browser, since in current state no browser is installed. I installed a firefox browser with the following command:

```python
sudo apt install firefox-esr
```

The other useful configuration is changing default text editor from emacs into Mousepad, for example.
You can do it by creating empty text file (by clicking right button on desktop Create Document/Empty File). Then click right button on that document and then Open With/Open With Other Application and then select Mousepad and additionally check box: Use as a default.

In other [article](/copy_paste_over_vnc.html) I describe how to enable copy paste over VNC between remote and local computers.